In [21]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar as data

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [22]:
tf.reset_default_graph()

<h1>Hyperparameter</h1>

In [28]:
epochs = 10;
batch_size = 128
lr = 0.01

<h1>Placeholder</h1>

In [29]:
tf.reset_default_graph()

# Define placeholders
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input')
y = tf.placeholder(tf.float32, shape=(None, 10), name='output')
d = tf.placeholder(tf.float32, name="droput_rate")

<h1>LeNet-5</h1>

In [30]:
def lenet5(x):
    
    ###### Convolution layer 6 @ 28x28 ######

    W1 = tf.Variable(tf.truncated_normal(shape=(5,5,3,6), mean = 0, stddev = 0.1))
    b1 = tf.Variable(tf.zeros(6))

    l1 = tf.nn.conv2d(x, W1, strides=[1,1,1,1], padding='VALID') + b1
    l1 = tf.nn.relu(l1)
    l1 = tf.nn.max_pool(l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

    ###### Convolution layer 16 @ 10x10 ######

    W2 = tf.Variable(tf.truncated_normal(shape=(5,5,6,16), mean = 0, stddev = 0.1))
    b1 = tf.Variable(tf.zeros(16))

    l2 = tf.nn.conv2d(l1, W2, strides=[1,1,1,1], padding='VALID')
    l2 = tf.nn.relu(l2)
    l2 = tf.nn.max_pool(l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

    ###### Fully connected (400 --> 120) ######
    flat = tf.contrib.layers.flatten(l2)

    W3 = tf.Variable(tf.truncated_normal(shape=(400,120), mean = 0, stddev = 0.1))
    b3 = tf.Variable(tf.zeros(120))

    l3 = tf.matmul(flat, W3) + b3
    l3 = tf.nn.relu(l3)

    ###### Fully connected (120 --> 84) ######
    W4 = tf.Variable(tf.truncated_normal(shape=(120,84), mean = 0, stddev = 0.1))
    b4 = tf.Variable(tf.zeros(84))

    l4 = tf.matmul(l3, W4) + b4
    l4 = tf.nn.relu(l4)

    ###### Fully connected (84 --> 10) ######
    W5 = tf.Variable(tf.truncated_normal(shape=(84,10), mean = 0, stddev = 0.1))
    b5 = tf.Variable(tf.zeros(10))
    
    logits = tf.matmul(l4, W5) + b5
    
    return logits 


<h1>Cost and Optimization</h1>

In [31]:
logits = lenet5(x)

#define loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y),name='loss')

#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(cost)

corr_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
acc = tf.reduce_mean(tf.cast(corr_pred, tf.float32))

def get_stats(session, cost, accuracy, b_feat, b_labels, v_feat, v_labels):
    loss = sess.run(cost,
                   feed_dict={
                       x: b_feat,
                       y: b_labels, 
                   })
    
    acc = sess.run(accuracy, 
                  feed_dict={
                      x: v_feat,
                      y: v_labels,
                  })

    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, acc))

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [32]:
#Initialize the variables
init = tf.global_variables_initializer()

v_feat, v_labels = pickle.load(open('preprocessed_validation.p', mode='rb'))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        
        n_batches = 5
        for batch_i in range(1, n_batches+1):
            for batch_x, batch_y in data.load_preprocessed_training_batch(batch_i, batch_size):
                sess.run(train_op, feed_dict={x:batch_x, y:batch_y})

            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
            get_stats(sess, cost, acc, batch_x, batch_y, v_feat, v_labels)

Epoch  1, CIFAR-10 Batch 1:  Loss:     2.3077 Validation Accuracy: 0.129400
Epoch  1, CIFAR-10 Batch 2:  Loss:     2.2784 Validation Accuracy: 0.157400
Epoch  1, CIFAR-10 Batch 3:  Loss:     2.2202 Validation Accuracy: 0.174800
Epoch  1, CIFAR-10 Batch 4:  Loss:     2.1867 Validation Accuracy: 0.191600
Epoch  1, CIFAR-10 Batch 5:  Loss:     2.0725 Validation Accuracy: 0.235000
Epoch  2, CIFAR-10 Batch 1:  Loss:     2.2587 Validation Accuracy: 0.247000
Epoch  2, CIFAR-10 Batch 2:  Loss:     2.0552 Validation Accuracy: 0.259600
Epoch  2, CIFAR-10 Batch 3:  Loss:     1.8299 Validation Accuracy: 0.254200
Epoch  2, CIFAR-10 Batch 4:  Loss:     1.9488 Validation Accuracy: 0.280200
Epoch  2, CIFAR-10 Batch 5:  Loss:     1.9832 Validation Accuracy: 0.295000
Epoch  3, CIFAR-10 Batch 1:  Loss:     2.2300 Validation Accuracy: 0.300200
Epoch  3, CIFAR-10 Batch 2:  Loss:     2.0040 Validation Accuracy: 0.302400
Epoch  3, CIFAR-10 Batch 3:  Loss:     1.6931 Validation Accuracy: 0.302200
Epoch  3, CI